# RNA-Seq Analysis Training Demo

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a prokaryotic data set. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression.

![RNA-Seq workflow](images/rnaseq-workflow.png)

In [ ]:
### STEP 1: Install Miniforge.

First install Miniforge. 

Miniforge is a tool that helps us install the Conda and Mamba package managers. Conda is a tool which creates Conda ‘environments’ that Conda packages can be installed into. 

Conda packages and environments are useful for several reasons. Conda packages contain metadata. This metadata includes information about what other programs the given software needs to be installed, in order to run. When installing a package with Conda, those other packages are automatically also installed. In this way, the user does not have to worry about manually installing each dependency. This makes installation quick and simple. 

These packages are installed inside of environments, which are simply folders within the local installation of Conda. This has several benefits. Local installation means easier installation for non-admin users who may not have access to all system directories. Each environment can hold specific software with specific versions, and it easy to swap to different environments. In addition, the environments themselves are portable, as each environment contains a manifest on how to recreate that environment.

Miniforge  is used to install and update Conda packages, which it gets from a ‘channel’, or repository. It is an alternative to the native Conda package manager. It is often used for reasons of speed. 

Bioconda is a ‘channel’, or repository, that the Mambaforge package manager can download packages from. It is a repository of Conda packages that are related to biology. These packages are versions of popular biology software that are curated and uploaded by contributing users. 


In [12]:
!curl -L -O "https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh"
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
100 81.7M  100 81.7M    0     0   213M      0 --:--:-- --:--:-- --:--:--  213M
PREFIX=/home/jupyter/miniforge
Unpacking bootstrapper...
Unpacking payload...
Extracting _libgcc_mutex-0.1-conda_forge.tar.bz2
Extracting ca-certificates-2025.11.12-hbd8a1cb_0.conda
Extracting libgomp-15.2.0-he0feb66_15.conda
Extracting nlohmann_json-abi-3.12.0-h0f90c79_1.conda
Extracting pybind11-abi-11-hc364b38_1.conda
Extracting python_abi-3.12-8_cp312.conda
Extracting tzdata-2025b-h78e105d_0.conda
Extracting _openmp_mutex-4.5-2_gnu.tar.bz2
Extracting libgcc-15.2.0-he0feb66_15.conda
Extracting bzip2-1.0.8-hda65f42_8.conda
Extracting c-ares-1.34.5-hb9d3cd8_0.conda
Extracting keyutils-1.6.3-hb9d3c

Next, using miniforge and bioconda, install the tools that will be used in this tutorial.

In [13]:
#tell the computer where the mambaforge bin files are located
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin"

#now we can easily use 'mamba' command to install software 
! mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc salmon

[+] 0.0s
[+] 0.1s
[+] 0.0s
[+] 0.1s
conda-forge/linux-64 ━━╸━━━━━━━━━━━━━━━╸━━━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
conda-forge/noarch   ━━━━╸━━━━━━━━━━━━━━━╸━━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
bioconda/linux-64    ━━━━━╸━━━━━━━━━━━━━━━╸━   0.0 B /  ??.?MB @  ??.?MB/s  0.0s
bioconda/noarch      ━━━━━━━━━━━━━━━━━━━━━━━  14.1kB /   5.1MB @ 196.7kB/s  0.0s[+] 0.2s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━   1.0MB /  49.9MB @   5.7MB/s  0.1s
conda-forge/noarch   ╸━━━━━━━━━━━━━━━━━━━━━━   1.2MB /  24.0MB @   6.6MB/s  0.1s
bioconda/linux-64    ━━━━━━━╸━━━━━━━━━━━━━━━   1.9MB /   5.4MB @  11.2MB/s  0.1s
bioconda/noarch      ━━━━━━━━━━━━━━━━╸━━━━━━   3.8MB /   5.1MB @  21.6MB/s  0.1s[+] 0.3s
conda-forge/linux-64 ━━━━━━━━━━━━━━━━━━━━━━━   2.2MB /  49.9MB @   9.5MB/s  0.2s
conda-forge/noarch   ━╸━━━━━━━━━━━━━━━━━━━━━   2.4MB /  24.0MB @  10.1MB/s  0.2s
bioconda/linux-64    ━━━━━━━━━━━━╸━━━━━━━━━━   3.2MB /   5.4MB @  13.8MB/s  0.2s
bioconda/noarch      ━━━━━━━━━━━━━━━━━━━━━╸━   5.0MB /   

### STEP 2: Setup Environment

Create a set of directories to store the reads, reference sequence files, and output files.


In [14]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data
!mkdir -p data/trunc_rawfastq
!mkdir -p data/trimmed
!mkdir -p data/fastqc
!mkdir -p data/reference

/home/jupyter/rnaseq-myco-notebook


Specify the number of available threads based on the VM.

This is useful for later tools such as trimmomatic, or salmon.

In [15]:
numthreads=!lscpu | grep '^CPU(s)'| awk '{print $2-1}'
THREADS = int(numthreads[0])
print("The number of threads is ", THREADS)

The number of threads is  3


### STEP 3: Copy FASTQ Files
In order for this tutorial to run quickly, we will only analyze 50,000 reads from a sample from both sample groupsinstead of analyzing all the reads from all six samples. These files have been posted on a Google Storage Bucket that we made publicly accessible.


In [16]:
! gsutil -m cp -r gs://rnaseq-myco-bucket/truncated-reads/* data/trunc_rawfastq/

Copying gs://rnaseq-myco-bucket/truncated-reads/SRR13349122_1.fastq...
Copying gs://rnaseq-myco-bucket/truncated-reads/SRR13349122_2.fastq...          
Copying gs://rnaseq-myco-bucket/truncated-reads/SRR13349128_1.fastq...          
Copying gs://rnaseq-myco-bucket/truncated-reads/SRR13349128_2.fastq...          
- [4/5 files][ 33.0 MiB/ 33.0 MiB]  99% Done                                    

### STEP 4: Copy reference transcriptome files that will be used by Salmon
Salmon is a tool that aligns RNA-Seq reads to a set of transcripts rather than the entire genome.

In [17]:
! gsutil -m cp -r gs://rnaseq-myco-bucket/reference/M_chelonae_transcripts.fasta data/reference/M_chelonae_transcripts.fasta
! gsutil -m cp -r gs://rnaseq-myco-bucket/reference/decoys.txt data/reference/decoys.txt

Copying gs://rnaseq-myco-bucket/reference/M_chelonae_transcripts.fasta...
/ [1/1 files][  9.4 MiB/  9.4 MiB] 100% Done                                    
Operation completed over 1 objects/9.4 MiB.                                      
Copying gs://rnaseq-myco-bucket/reference/decoys.txt...
/ [1/1 files][   14.0 B/   14.0 B] 100% Done                                    
Operation completed over 1 objects/14.0 B.                                       


### STEP 5: Copy data file for Trimmomatic

In [18]:
! gsutil -m cp -r gs://rnaseq-myco-bucket/reference/TruSeq3-PE.fa .

Copying gs://rnaseq-myco-bucket/reference/TruSeq3-PE.fa...
/ [1/1 files][   95.0 B/   95.0 B] 100% Done                                    
Operation completed over 1 objects/95.0 B.                                       


### STEP 6: Run Trimmomatic
Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Trimmomatic takes an input, in this case a forward and reverse FASTQ file, and the user names the outputs. In this case the first two outputs are the ‘paired’ and ‘orphaned’ trimmed files for the forward reads. The second two are the ‘paired’ and ‘orphaned’ trimmed files for the reverse reads. 

Paired in this case means the forward and reverse sequences were able to be aligned to each other after trimming. Orphaned, or unpaired, typically means one of the reads was discarded as a result of trimming, and only the forward or reverse read survived. 

Here we take only the file with paired reads, as there are only a few orphaned reads, and including orphaned reads can complicate different downstream analyses. Unless there is a significant amount of them, or a specific reason to use them, it is generally easier to discard unpaired reads. Also in the interest of simplicity and speed, we proceed in further steps with just using the paired-end reads of just the forward-end read files, which in this context is sufficient, however in different contexts, using both forward and reverse is often preferable. 

The last part of the command specifies how the trimming is done:
ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

In greater detail:

‘ILLUMINACLIP:TruSeq3-PE.fa’ refers to which adapters should be cut from the reads. 

‘2:30:10:2’ refers to various metrics, which are recommended defaults. 

‘2’ refers to the seed mismatch. This refers to the amount of mismatches a 'seed' may have in aligning to a possible adapter.

‘30’ refers to the palindrome clip threshold. This refers to the similarity score. If forward and reverse reads, after having an adapter attached to them, are greater than this score, trimming of adapter fragments will be performed. Forward reads are clipped, and reverse reads dropped.

‘10’ refers to the simple clip threshold. The basic, and alternative method to palindromic searching for adapters. Adapters are tested against reads and if sufficiently matched (above the threshold, in this case, 10), they are clipped. 

‘2’ refers to the minimum adapter fragment length in palindrome mode. 

‘LEADING:3’ refers to trimming bases from the start of a read. Bases at the start of a read will continue to be trimmed, sequentially, as long as the bases remain below a PHRED score of 3.

‘TRAILING:3’ refers to the same as above, but at the end of a read.

‘MINLEN:36’, the read is discarded if below this length.

Greater information about parameters can be found in the trimmomatic documentation. 

In [19]:
! trimmomatic PE -threads $THREADS data/trunc_rawfastq/SRR13349122_1.fastq data/trunc_rawfastq/SRR13349122_2.fastq data/trimmed/SRR13349122_1_trimmed.fastq data/trimmed/SRR13349122_1_trimmed_unpaired.fastq data/trimmed/SRR13349122_2_trimmed.fastq  data/trimmed/SRR13349122_2_trimmed_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
! trimmomatic PE -threads $THREADS data/trunc_rawfastq/SRR13349128_1.fastq data/trunc_rawfastq/SRR13349128_2.fastq data/trimmed/SRR13349128_1_trimmed.fastq data/trimmed/SRR13349128_1_trimmed_unpaired.fastq data/trimmed/SRR13349128_2_trimmed.fastq  data/trimmed/SRR13349128_2_trimmed_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36


TrimmomaticPE: Started with arguments:
 -threads 3 data/trunc_rawfastq/SRR13349122_1.fastq data/trunc_rawfastq/SRR13349122_2.fastq data/trimmed/SRR13349122_1_trimmed.fastq data/trimmed/SRR13349122_1_trimmed_unpaired.fastq data/trimmed/SRR13349122_2_trimmed.fastq data/trimmed/SRR13349122_2_trimmed_unpaired.fastq ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
ILLUMINACLIP: Using adapter file from Trimmomatic installation folder: /opt/conda/share/trimmomatic-0.40-0/adapters/TruSeq3-PE.fa
Using PrefixPair: 'TACACTCTTTCCCTACACGACGCTCTTCCGATCT' and 'GTGACTGGAGTTCAGACGTGTGCTCTTCCGATCT'
ILLUMINACLIP: Using 1 prefix pairs, 0 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Read Pairs: 50000 Both Surviving: 49870 (99.74%) Forward Only Surviving: 130 (0.26%) Reverse Only Surviving: 0 (0.00%) Dropped: 0 (0.00%)
TrimmomaticPE: Completed successfully
TrimmomaticPE: Started with arguments:
 -thr

### STEP 7: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

Because jupyter is at its core a python editor, we can use python code and html support to display results in-line.

FastQC looks for different characteristics of quality in reads. It is very rare that every metric will pass. In many cases, they serve as warnings, which should be compared to the context of the experiment. For instance, here, per base sequence content, sequence length distribution, sequence duplication levels, and overrepresented sequences all throw warnings. Per base sequence content routinely fails in RNA-sequencing in the first 15~ or so bases due to biased fragmentation. In most of our samples, this is where we see the failure (20% or more difference between A/T or G/C), and so is not unexpected. The overrepresented sequences can be BLASTed to show the majority of them are ribosomal RNA. Ribosomal RNA contamination is also common and will not be indexed later, and so not a large concern. Other metrics look good.

In [20]:
! fastqc -o data/fastqc data/trimmed/SRR13349122_1_trimmed.fastq
! fastqc -o data/fastqc data/trimmed/SRR13349128_1_trimmed.fastq

from IPython.display import IFrame
IFrame(src='./data/fastqc/SRR13349122_1_trimmed_fastqc.html', width=800, height=600)

null
Started analysis of SRR13349122_1_trimmed.fastq
Approx 5% complete for SRR13349122_1_trimmed.fastq
Approx 10% complete for SRR13349122_1_trimmed.fastq
Approx 15% complete for SRR13349122_1_trimmed.fastq
Approx 20% complete for SRR13349122_1_trimmed.fastq
Approx 25% complete for SRR13349122_1_trimmed.fastq
Approx 30% complete for SRR13349122_1_trimmed.fastq
Approx 35% complete for SRR13349122_1_trimmed.fastq
Approx 40% complete for SRR13349122_1_trimmed.fastq
Approx 45% complete for SRR13349122_1_trimmed.fastq
Approx 50% complete for SRR13349122_1_trimmed.fastq
Approx 55% complete for SRR13349122_1_trimmed.fastq
Approx 60% complete for SRR13349122_1_trimmed.fastq
Approx 65% complete for SRR13349122_1_trimmed.fastq
Approx 70% complete for SRR13349122_1_trimmed.fastq
Approx 75% complete for SRR13349122_1_trimmed.fastq
Approx 80% complete for SRR13349122_1_trimmed.fastq
Approx 85% complete for SRR13349122_1_trimmed.fastq
Approx 90% complete for SRR13349122_1_trimmed.fastq
Approx 95% c

### STEP 8: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

Being able to use python with bash also means we can seamlessly use popular python packages, such as pandas, to interact with or view the files we create.

In [21]:
! multiqc -f data/fastqc

import pandas as pd
dframe = pd.read_csv("./multiqc_data/multiqc_fastqc.txt", sep='\t')
display(dframe)


/// ]8;id=905478;https://multiqc.info\MultiQC]8;;\ 🍾 v1.33

       file_search | Search path: /home/jupyter/rnaseq-myco-notebook/data/fastqc
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 4/4  0/4  
            fastqc | Found 2 reports
     write_results | Data        : multiqc_data
     write_results | Report      : multiqc_report.html
           multiqc | MultiQC complete


,Sample,Filename,File type,Encoding,Total Sequences,Total Bases,Sequences flagged as poor quality,Sequence length,%GC,total_deduplicated_percentage,...,basic_statistics,per_base_sequence_quality,per_sequence_quality_scores,per_base_sequence_content,per_sequence_gc_content,per_base_n_content,sequence_length_distribution,sequence_duplication_levels,overrepresented_sequences,adapter_content
0,SRR13349122_1,SRR13349122_1_trimmed.fastq,Conventional base calls,Sanger / Illumina 1.9,49870.0,2.5 Mbp,0.0,50-51,55.0,19.153800,...,pass,pass,pass,fail,pass,pass,warn,fail,fail,pass
1,SRR13349128_1,SRR13349128_1_trimmed.fastq,Conventional base calls,Sanger / Illumina 1.9,49851.0,2.5 Mbp,0.0,50-51,54.0,16.914405,...,pass,pass,pass,fail,warn,pass,warn,fail,fail,pass


### STEP 9: Index the Transcriptome so that Trimmed Reads Can Be Mapped Using Salmon

Indexing is a method of calculating readcounts. A reference transcriptome is used to build an index. The primary purpose of the index is to quickly and efficiently match read sequences to specific gene entries. The technical aspect of how this is done is in the salmon documentation, but it utilizes ‘quasi-mapping’, which includes hashing of kmers. 

In [22]:
! salmon index -t data/reference/M_chelonae_transcripts.fasta -p $THREADS -i data/reference/transcriptome_index --decoys data/reference/decoys.txt -k 31 --keepDuplicates


Version Server Response: Not Found
index ["data/reference/transcriptome_index"] did not previously exist  . . . creating it
[2026-01-05 17:56:50.538] [jLog] [info] building index
out : data/reference/transcriptome_index
[2026-01-05 17:56:50.538] [puff::index::jointLog] [info] Running fixFasta

[Step 1 of 4] : counting k-mers

[2026-01-05 17:56:50.773] [puff::index::jointLog] [warning] There were 2 transcripts that would need to be removed to avoid duplicates.
[2026-01-05 17:56:50.774] [puff::index::jointLog] [info] Replaced 0 non-ATCG nucleotides
[2026-01-05 17:56:50.774] [puff::index::jointLog] [info] Clipped poly-A tails from 0 transcripts
wrote 4868 cleaned references
[2026-01-05 17:56:50.791] [puff::index::jointLog] [info] Filter size not provided; estimating from number of distinct k-mers
[2026-01-05 17:56:50.894] [puff::index::jointLog] [info] ntHll estimated 4966944 distinct k-mers, setting filter size to 2^27
Threads = 3
Vertex length = 31
Hash functions = 5
Filter size = 13421

### STEP 10: Run Salmon to Map Reads to Transcripts and Quantify Expression Levels
Salmon aligns the trimmed reads to the reference transcriptome and generates the read counts per transcript. In this analysis, each gene has a single transcript.

-quant indicates using the quantification, or read-count tool of Salmon. 

-i indicates the location of the index

-r refers to the input read, or FASTQ, files. 

-p is used for parallel processing, and indicates the amount of CPUs or CPU threads. 

-o indicates the output folder name, and prefix name for your files, that you would like.


In [23]:
! salmon quant -i data/reference/transcriptome_index -l SR -r data/trimmed/SRR13349122_1_trimmed.fastq -p $THREADS --validateMappings -o data/quants/SRR13349122_quant
! salmon quant -i data/reference/transcriptome_index -l SR -r data/trimmed/SRR13349128_1_trimmed.fastq -p $THREADS --validateMappings -o data/quants/SRR13349128_quant


Version Server Response: Not Found
### salmon (selective-alignment-based) v1.10.3
### [ program ] => salmon 
### [ command ] => quant 
### [ index ] => { data/reference/transcriptome_index }
### [ libType ] => { SR }
### [ unmatedReads ] => { data/trimmed/SRR13349122_1_trimmed.fastq }
### [ threads ] => { 3 }
### [ validateMappings ] => { }
### [ output ] => { data/quants/SRR13349122_quant }
Logs will be written to data/quants/SRR13349122_quant/logs
[2026-01-05 17:56:58.023] [jointLog] [info] setting maxHashResizeThreads to 3
-----------------------------------------
| Loading contig table | Time = 1.1102 ms
-----------------------------------------
[2026-01-05 17:56:58.023] [jointLog] [info] Fragment incompatibility prior below threshold.  Incompatible fragments will be ignored.
[2026-01-05 17:56:58.023] [jointLog] [info] Usage of --validateMappings implies use of minScoreFraction. Since not explicitly specified, it is being set to 0.65
[2026-01-05 17:56:58.023] [jointLog] [info] Sett

### STEP 11: Report the top 10 most highly expressed genes in the samples.

The quant files can be opened with any general text editor. Here we use some quick linux commands to examine the top 10 genes in quant files.

Sort is a linux shell command. The n, r, and k commands 
sort -nrk 5,5 

-n means to sort numerically

-r means to sort in reverse order, from largest to smallest

-k means to sort based on a specific field. In this case, the fifth column.

-the final 5 means to show only the top five entries.

Top 10 most highly expressed genes in the wild-type sample. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`


In [24]:
#Note, the 'write failed error' is not really an error and perfectly fine.
! sort -nrk 5,5 data/quants/SRR13349122_quant/quant.sf | head -10

BB28_RS20665	1293	1043.000	5447.071698	55.000
BB28_RS03075	1626	1376.000	3077.869008	41.000
BB28_RS07370	9255	9005.000	424.426717	37.000
BB28_RS20690	10377	10127.000	367.203150	36.000
BB28_RS19405	3948	3698.000	1005.588509	36.000
BB28_RS18585	1305	1055.000	2741.512828	28.000
BB28_RS20685	7731	7481.000	372.811085	27.000
BB28_RS19310	1194	944.000	2626.176789	24.000
BB28_RS22260	1836	1586.000	1497.991546	23.000
BB28_RS09805	1437	1187.000	2001.528725	23.000
sort: write failed: 'standard output': Broken pipe
sort: write error


Top 10 most highly expressed genes in the double lysogen sample.


In [25]:
! sort -nrk 5,5 data/quants/SRR13349128_quant/quant.sf | head -10

BB28_RS20665	1293	1043.000	16640.051824	107.000
BB28_RS18585	1305	1055.000	5381.096618	35.000
BB28_RS13330	2790	2540.000	2107.343957	33.000
BB28_RS14905	972	722.000	6065.711819	27.000
BB28_RS22260	1836	1586.000	1943.146845	19.000
BB28_RS18315	1767	1517.000	2031.529926	19.000
BB28_RS20685	7731	7481.000	368.590781	17.000
BB28_RS20690	10377	10127.000	240.251247	15.000
BB28_RS03075	1626	1376.000	1768.186333	15.000
BB28_RS11085	1278	1028.000	2208.971570	14.000
sort: write failed: 'standard output': Broken pipe
sort: write error


### STEP 12: Report the expression of a putative acyl-ACP desaturase (BB28_RS16545) that was downregulated in the double lysogen relative to wild-type
A acyl-transferase was reported to be downregulated in the double lysogen as shown in the table of the top 20 upregulated and downregulated genes from the paper describing the study.
![RNA-Seq workflow](images/table-cushman.png)

Use `grep` to report the expression in the wild-type sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [26]:
! grep 'BB28_RS16545' data/quants/SRR13349122_quant/quant.sf

BB28_RS16545	987	737.000	560.631139	4.000


Use `grep` to report the expression in the double lysogen sample. The fields in the Salmon `quant.sf` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
`Name    Length  EffectiveLength TPM     NumReads`

In [27]:
! grep 'BB28_RS16545' data/quants/SRR13349128_quant/quant.sf

BB28_RS16545	987	737.000	220.083619	1.000


## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.

In [ ]:

[Workflow One:](Tutorial_1.ipynb) A short introduction to downloading and mapping sequences to a transcriptome using Trimmomatic and Salmon. Here is a link to the YouTube video demonstrating the tutorial: <https://youtu.be/ChGfBR4do_Y>.

[Workflow One (Extended):](Tutorial_1B_Extended.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset, and includes elaboration such as using SRA tools for sequence downloading, and examples of running batches of fastq files through the pipeline. This workflow may take around an hour to run.

[Workflow One (Using Snakemake):](Tutorial_2_Snakemake.ipynb) Using snakemake to run workflow one.

[Workflow Two (DEG Analysis):](Tutorial_3_DEG_Analysis.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.


![RNA-Seq workflow](images/RNA-Seq_Notebook_Homepage.png)